In [1]:
import requests
from dotenv import load_dotenv
import os
import numpy as np
from langchain_chroma import Chroma

In [2]:
headers = {
    "authorization": os.getenv("myserver_api_key")
}

embed_url = os.getenv("myserver_url")+":8000/embed"
rerank_url = os.getenv("myserver_url")+":8000/rerank"

In [3]:
temp_txt_list = ["test", "bitch"]
payload = {
    "texts": temp_txt_list,
    "is_query": False
}
response = requests.post(embed_url, json=payload, headers=headers)
response

<Response [200]>

In [4]:
embeddings_array = np.array(response.json()["embeddings"])
embeddings_array.shape

(2, 2560)

In [5]:
embeddings_array

array([[-6.77108765e-05,  4.19921875e-02, -8.30078125e-03, ...,
        -2.45361328e-02,  1.94091797e-02, -1.47094727e-02],
       [-1.64031982e-04,  1.81884766e-02,  3.68652344e-02, ...,
         9.39941406e-03,  5.72204590e-04,  3.47900391e-03]],
      shape=(2, 2560))

In [6]:
class CustomEmbedding:
    def embed_documents(self, texts):
        payload = {
            "texts": texts,
            "is_query": False
        }
        return requests.post(embed_url, json=payload, headers=headers).json()["embeddings"]
    
    def embed_query(self, text):
        payload = {
            "texts": [text],
            "is_query": True
        }
        return requests.post(embed_url, json=payload, headers=headers).json()["embeddings"][0]

In [7]:
embeddings = CustomEmbedding()

In [8]:
vector_store = Chroma.from_texts(
    texts=temp_txt_list, embedding=embeddings, persist_directory="../chroma_db"
)

In [9]:
# mmr 기반
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":50}
)

In [10]:
docs = retriever.get_relevant_documents("JVM에 대해 알려줘")
docs

C:\Users\user1\AppData\Local\Temp\ipykernel_10732\3866927720.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("JVM에 대해 알려줘")


[Document(id='98f4ce62-07ba-4bcc-b982-b926e4c3f0c6', metadata={}, page_content='test'),
 Document(id='57b948c6-3938-4e9e-89c0-746000a26bbe', metadata={}, page_content='bitch')]

In [13]:
vector_store = Chroma(
    persist_directory="../chroma_db",
    embedding_function=embeddings
)

In [14]:
# mmr 기반
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":50}
)

In [15]:
docs = retriever.get_relevant_documents("JVM에 대해 알려줘")
docs

[Document(id='98f4ce62-07ba-4bcc-b982-b926e4c3f0c6', metadata={}, page_content='test'),
 Document(id='57b948c6-3938-4e9e-89c0-746000a26bbe', metadata={}, page_content='bitch')]